In [3]:
#toy model bayesian signature inference

import torch
import pyro
import pyro.distributions as dist 
import numpy as np

#define the model in terms of M = phylogenetic signature matrix, K_denovo = number of signatures we want to infer de novo,
# K_fixed = number of signatures we already know. The fixed signatures are described by the fixed matrix Beta_fixed. 
# We provide an adjacency matrix A with binary entries which specify the correlation structure imposed by the phylogeny. 
# K = K_denovo + K_fixed
# M has dims (num_branches,96)
# theta is a vector encoding the number of mutations for each branch
# beta prior and alpha prior provide densities for the normal priors for Beta_denovo and alpha
# beta prior has dims (K_denovo,96), alpha prior has dims (num_branches,K_denovo)


def model_single_run(M,beta_fixed,K_denovo,beta_prior,alpha_prior):
    
    num_samples = M.size()[0]
    
    K_fixed = beta_fixed.size()[0]
    
    theta = torch.sum(M,axis = 1)
    
    #parametrize the activity matrix as theta*alpha, where theta encodes the total number of mutations of the branches 
    # and alpha's are percentages of signature activity
    
   # sample alpha from a normal distribution using alpha prior

    alpha = pyro.sample("activities", dist.Normal(alpha_prior,1))
    
    # sample the extra signature profiles from normal distribution
    
    beta_denovo = pyro.sample("extra_signatures", dist.Normal(beta_prior,1))  
    
    # enforce non negativity
    
    alpha = torch.exp(alpha)
    
    beta_denovo = torch.exp(beta_denovo)
    
    #normalize
    
    alpha = alpha/(torch.sum(alpha, 1).unsqueeze(-1))
    
    beta_denovo = beta_denovo/(torch.sum(beta_denovo,1).unsqueeze(-1))
    
    # build full beta matrix

    beta = torch.cat((beta_fixed,beta_denovo), axis = 0)
    
    # write the likelihood

    with pyro.plate("context",96):
        
        with pyro.plate("sample",num_samples):
    
            pyro.sample("obs", dist.Poisson(torch.matmul(torch.matmul(torch.diag(theta),alpha),beta)), obs = M)
        

In [13]:
from pyro.infer.autoguide import AutoDelta
from pyro.infer.autoguide.initialization import init_to_sample
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam  
import matplotlib.pyplot as plt


# perform MAP estimates
 
def guide_single_run(M,beta_fixed,K_denovo,beta_prior,alpha_prior):
    
    alpha = pyro.param("alpha", dist.Normal(alpha_prior,1).sample())
    
    beta = pyro.param("beta", dist.Normal(beta_prior,1).sample())
    
    pyro.sample("activities", dist.Delta(alpha))
    
    pyro.sample("extra_signatures", dist.Delta(beta)) 
    
    
# SVI

def inference_single_run(M,beta_fixed,K_denovo,beta_prior,alpha_prior,lr=0.05,num_steps = 200):

    pyro.clear_param_store()  # always clear the store before the inference

    # learning global parameters

    adam_params = {"lr": lr}
    optimizer = Adam(adam_params)
    elbo = Trace_ELBO()

    svi = SVI(model_single_run, guide_single_run, optimizer, loss=elbo)

#   inference

#   do gradient steps

    for step in range(num_steps):
        
        loss = svi.step(M,beta_fixed,K_denovo,beta_prior,alpha_prior) 

In [14]:
def calculate_transfer_coeff(parameters,M,beta_fixed,A,hyper_lambda):
    
    
    alpha = torch.exp(parameters["alpha"])
    alpha = alpha/(torch.sum(alpha,1).unsqueeze(-1))
    
    beta_denovo = torch.exp(parameters["beta"])
    beta_denovo = beta_denovo/(torch.sum(beta_denovo,1).unsqueeze(-1))
    
    beta = torch.cat((beta_fixed,beta_denovo),axis = 0)
    
    theta = torch.sum(M,axis = 1)
    
    num_samples = M.size()[0]
    
    cos = torch.zeros(num_samples,num_samples)
    
    for i in range(num_samples):
        
        for j in range(num_samples):
        
            if A[i,j] == 1:
                
                M_r = theta[i]*torch.matmul(alpha[j],beta)
                
                cos[i,j] = torch.dot(M[i],M_r)/(torch.norm(M[i])*torch.norm(M_r))
                
    
    
    w = cos/(torch.sum(cos,1).unsqueeze(-1))
    
    transfer_coeff = torch.zeros(num_samples,num_samples)
    
    
    for i in range(num_samples):
        
        for j in range(num_samples):
            
            if i==j:
                
                transfer_coeff[i,j]  = (1-hyper_lambda)*w[i,j]
            
            else:
                
                transfer_coeff[i,j]  = hyper_lambda*w[i,j]
     
    
    return transfer_coeff

In [33]:
def full_inference(M,A,beta_fixed,K_denovo,hyper_lambda,lr = 0.05,steps_per_iteration = 200,num_iterations = 10):


    # first indipendent run
    
    num_samples = M.size()[0]
    
    K_fixed = beta_fixed.size()[0]
            
    # step 0 : indipendent inference
    
    print("iteration ",0)
            
    alpha_prior = dist.Normal(torch.zeros(num_samples,K_denovo + K_fixed),1).sample()
            
    beta_prior = dist.Normal(torch.zeros(K_denovo,96),1).sample()

    inference_single_run(M,beta_fixed,K_denovo,beta_prior,alpha_prior,lr = lr, 
                                            num_steps = steps_per_iteration)
    
    
    # do iterations transferring alpha's
            
    for i in range(num_iterations):    
                    
        print("iteration ", i+1)
            
             
            #extract inferred alpha's and beta'a from pyro store
            
        parameters={}
        
        
        for key in pyro.get_param_store().get_all_param_names() :
                
            parameters.update({key : torch.tensor(pyro.param(key))})
                
        
        alpha_prior = parameters["alpha"]
        
        beta_prior = parameters["beta"]
            
       
       #calculate transfer coeff
        
        transfer_coeff = calculate_transfer_coeff(parameters,M,beta_fixed,A,hyper_lambda)
            
            
            #update alpha prior with transfer coeff
            
        alpha_prior = torch.matmul(transfer_coeff,alpha_prior)
            
            
            # do inference with updates alpha_prior and beta_prior
    
        inference_single_run(M,beta_fixed,K_denovo,beta_prior,alpha_prior,lr = lr, 
                                            num_steps = steps_per_iteration)
    
            
    #save final inference

    alpha_denovo = torch.exp(parameters["alpha"])
    alpha_denovo = alpha_denovo/(torch.sum(alpha_denovo,1).unsqueeze(-1))
    
    beta_denovo = torch.exp(parameters["beta"])
    beta_denovo = beta_denovo/(torch.sum(beta_denovo,1).unsqueeze(-1))
    
    SigPhylo = {"alpha" : alpha_denovo, "beta" : beta_denovo, "lambda" : hyper_lambda, "K de novo": K_denovo}
                
       
    return SigPhylo            

In [34]:
#load data

my_path = "/Users/riccardobergamin/Documents/GitHub/SigPhylo/toy_model/"
import pandas as pd

# load data
X_1 = pd.read_csv(my_path + "sample_1.csv")
X_2 = pd.read_csv(my_path + "sample_2.csv")
c_1 = X_1.values[:,1:]
c_2 = X_2.values[:,1:]
M_1 = torch.tensor(np.array(c_1,dtype=float))
M_2 = torch.tensor(np.array(c_2,dtype=float))
M_1 = M_1.float()
M_2 = M_2.float()
# load clock-like signatures
clock_like_signatures = pd.read_csv(my_path + "beta_aging.csv")
beta_fixed = clock_like_signatures.values[:,1:]
beta_fixed = torch.tensor(np.array(beta_fixed,dtype=float))
beta_fixed = beta_fixed.float()
# define adjacency matrix
A = torch.ones(3,3)

In [28]:
# sample 1

full_inference(M_1,A,beta_fixed,K_denovo=1,hyper_lambda=0.5,lr = 0.005,steps_per_iteration = 500,num_iterations = 30)

iteration  0
iteration  1
alpha tensor([-2.5893, -0.4575,  1.5708])


<ipython-input-21-fdad0f204f1c>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  parameters.update({key : torch.tensor(pyro.param(key))})


iteration  2
alpha tensor([-0.9305, -1.1592,  1.9921])
iteration  3
alpha tensor([-0.4993, -2.4958,  2.3085])
iteration  4
alpha tensor([-1.4134, -1.3559,  2.4939])
iteration  5
alpha tensor([-1.3471, -2.5649,  0.7761])
iteration  6
alpha tensor([-0.8285, -0.4652,  1.9562])
iteration  7
alpha tensor([-2.1639, -1.2003,  1.7462])
iteration  8
alpha tensor([-1.0960, -0.3752,  2.5205])
iteration  9
alpha tensor([-1.2584, -1.7445,  1.2285])
iteration  10
alpha tensor([-0.9742, -0.8802,  2.1420])
iteration  11
alpha tensor([-1.0866, -0.9796,  1.9942])
iteration  12
alpha tensor([-1.2319, -0.4938,  2.6316])
iteration  13
alpha tensor([-0.6621, -0.5165,  2.4695])
iteration  14
alpha tensor([-2.2499, -1.0505,  1.6626])
iteration  15
alpha tensor([-0.5546, -0.5876,  2.4214])
iteration  16
alpha tensor([-1.3920, -0.4873,  2.4530])
iteration  17
alpha tensor([-1.7748, -0.4154,  1.9103])
iteration  18
alpha tensor([-1.3085,  0.0162,  2.7495])
iteration  19
alpha tensor([-2.1197, -1.4670,  1.6483])


{'alpha': tensor([[0.2343, 0.6640, 0.1017],
         [0.0356, 0.1681, 0.7963],
         [0.0312, 0.1378, 0.8310]]),
 'beta': tensor([[1.0648e-01, 4.5214e-03, 3.2899e-04, 8.6477e-03, 1.7579e-03, 2.8735e-06,
          8.0393e-04, 1.1044e-03, 2.0043e-05, 5.9519e-07, 4.6585e-07, 5.9171e-06,
          4.9833e-03, 3.8154e-03, 3.6517e-03, 6.4200e-03, 7.2760e-07, 3.5336e-06,
          1.3584e-06, 1.3621e-06, 4.1613e-05, 1.3985e-07, 1.7607e-06, 7.8309e-06,
          3.4872e-01, 2.3027e-03, 2.5877e-03, 9.6920e-03, 1.5340e-04, 1.0096e-05,
          1.1525e-05, 3.0896e-04, 2.1642e-06, 1.9043e-05, 9.2680e-07, 1.6290e-04,
          2.6317e-03, 7.5643e-04, 1.5227e-03, 3.5926e-03, 5.7332e-05, 4.9895e-04,
          5.4042e-06, 6.8096e-06, 2.1913e-05, 3.4003e-06, 9.5401e-05, 5.7111e-06,
          2.0877e-01, 1.2689e-02, 1.0903e-02, 6.4838e-02, 3.1219e-03, 3.0251e-04,
          2.1949e-05, 1.5593e-03, 1.5050e-03, 3.5424e-06, 2.6769e-07, 8.0531e-05,
          7.4389e-03, 8.3791e-03, 8.9112e-03, 3.9443e-03